# FPI Report and Nifty Indices,Exploratory Data Analysis


- The goal of this notebook is to do data analysis on NSDL Sector-wise FPI Investment report, as well as the sectoral Indices.

### Dataset
- Dataset consists of FPIs report and Nifty indices data from kaggle.


- Lets quicky understand what those are:

#### What is FPI?
- Foreign Investment inflow is an important reason for India’s economic growth. So to simplify compliance requirements and have uniform guidelines for various categories of foreign investors like Foreign Institutional Investors (FIIs), Sub Accounts and Qualified Foreign Investors (QFIs) merged into a new investor class termed as Foreign Portfolio Investors (FPIs).


- SEBI has authorized NSDL to monitor of these Group investment and various data related to FPI activities to be displayed on NSDL web portal.


- NSDL provides this data at an interval of 15 days.

*Source: [FPI NSDL](https://www.fpi.nsdl.co.in/web/Reports/FPI_Fortnightly_Selection.aspx)*

#### Nifty sectorial Indices
- This includes NIFTY 50, NIFTY AUTO, NIFTY BANK, NIFTY FMCG, NIFTY IT, NIFTY METAL, NIFTY OILGAS, NIFTY PHARMA, NIFTY PRIVATE BANK.


- These indices are designed to reflect the behavior and performance of their respective sectors.

### Tools used
- Pandas
- Datetime
- glob
- plotly

## Downloading the Dataset

- Get nifty indices data from kaggle.
*Source: [INDICES DATA](https://www.kaggle.com/atrisaxena/nifty-indices-data)*

- Installing necessary packages 

In [1]:
!pip install jovian openpyxl cufflinks plotly opendatasets --upgrade --quiet

Let's begin by downloading the data, and listing the files within the dataset.

In [2]:
# storing kaggle dataset url in a varaible
dataset_url = 'https://www.kaggle.com/atrisaxena/nifty-indices-data'

In [6]:
# download data from url
import opendatasets as od
od.download(dataset_url)

Skipping, found downloaded files in "./nifty-indices-data" (use force=True to force download)


In [7]:
# get FPI data from github
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


# function to download and unzip files
def download_and_unzip(url, extract_to='./FPI_Data'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

In [8]:
download_and_unzip('https://github.com/doke93/FPI_EDA/files/8159061/FPI_Data.zip')

- Store data path in variable

In [9]:
index_data_dir = './nifty-indices-data'
fpi_data_dir = './FPI_Data'

In [10]:
# check whether data is loaded in the notebook
import os
os.listdir(fpi_data_dir)[-1]

'FPI_30-Sep-2021.xlsx'

In [11]:
project_name = "fpi-indices-data-analysis"

In [12]:
jovian.commit(project=project_name)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "dokeabhishek3/fpi-indices-data-analysis" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis


'https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis'

### Importing libraries

In [13]:
import pandas as pd
import glob
import datetime
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

## Data Preparation and Cleaning

- Creating pandas dataframe by merging all the FPIs reports.

In [14]:
# function to read xlsx file, remove columns that are not relevant for our analysis. 
def get_data(xlsx_file):
    temp = pd.read_excel(xlsx_file, sheet_name='Sheet1')
    df = temp[2:-1].copy()
    
    # Select subset of columns with the relevant data for our analysis
    df = df.drop(df.columns[createList(0,(len(df.columns)-1))], axis = 1)
    df.rename(columns={'Unnamed: 1': 'Sector'},inplace=True)
    return df

# function to create a list of numbers
def createList(r1, r2):
    return [item for item in range(r1, r2+1) if (item != 1)&(item != 2)& (item != 32)]

# fetch all the .xlsx files from the diretory
path = pd.DataFrame(glob.glob(fpi_data_dir + "/*.xlsx"),columns=['location'])

# Parse dates from the column name
path['data_date'] = path['location'].apply(lambda x: x.split('/')[2].split('_')[1].split('.')[0])
path['data_date'] = path['data_date'].apply(lambda x: datetime.datetime.strptime(x,'%d-%b-%Y'))

# sort the data as per data_date in ascending order
path.sort_values(['data_date'], inplace=True)
path.reset_index(drop=True, inplace=True)

# for loop to store fpi data in a key and value pair and latter merging the data 
my_dict = {}
i = 1
for index, row in path.iterrows():
    my_dict[f"df_{i}"] = get_data(row['location'])
    
    for c in my_dict[f"df_{i}"].columns[1:]:
        col_name = c.split(' ')[3:]
        col = ''.join(col_name)
        my_dict[f"df_{i}"].rename(columns={c:col},inplace=True)
    
    if len(my_dict)==1:
        merged_df = my_dict[f"df_{i}"]
    else:
        merged_df = pd.merge(merged_df, my_dict[f"df_{i}"], on="Sector")
    i += 1

fpi_df = merged_df.copy()
fpi_df['Sector'] = fpi_df['Sector'].str.replace(" ","_")
fpi_df.set_index('Sector', inplace=True)

#sort data as per index (in alphabetical order)
fpi_df.sort_index(axis = 0, inplace=True)

- Creating dictionary object containing five consecutive period data.

In [15]:
subset_df = {}
for i in range(0,len(fpi_df.columns)):
    start = i
    if i < len(fpi_df.columns)-5:
        end = i + 5
        subset_df[f"{fpi_df.columns[start]}:{fpi_df.columns[end]}"] = fpi_df[fpi_df.columns[start:end]]

- Calculating consecutive column difference of FPIs to find out, how much the fund allocation for each sector has changed for every 15 days.


- Further calculate average for the past four period in each iteration and save it in a fpi_avg_df.

In [16]:
diff_df = {}
for key in subset_df.keys():
    diff_df[key] = subset_df[key].diff(periods=1,axis=1)
    diff_df[key][f"Average_for_{key.split(':')[1]}"]=diff_df[key].iloc[:,1:].mean(axis=1)
    diff_df[key].reset_index(inplace=True)
    if len(diff_df)==1:
        fpi_avg_df = diff_df[key].drop(diff_df[key].columns[[1,2,3,4,5]], axis=1)
    else:
        fpi_avg_df = pd.merge(fpi_avg_df, diff_df[key].drop(diff_df[key].columns[[1,2,3,4,5]], axis=1), on='Sector')    

- Creating a dictionary object for index data to make data handling easier in the later stage.


- Combining indices closing price in a single dataframe

In [17]:
index_dict = {}
for i in os.listdir(index_data_dir):
    key = i.split('.')[0].replace(" ","_")
    index_dict[key] = pd.read_csv(index_data_dir+f'/{i}', parse_dates=['Date'])
    try:
        index_dict[key].drop(['P/E','P/B', 'Div Yield','Turnover'], axis=1, inplace=True)
    except:
        index_dict[key].drop(['Shares Traded','Turnover (Rs. Cr)'], axis=1, inplace=True)
        
    if len(index_dict)==1:
        index_close_df = index_dict[key][['Date','Close']]
        index_close_df.rename(columns={'Close':key},inplace=True)
    elif key!='NIFTY_SMALLCAP_250':
        index_close_df = pd.merge(index_close_df, index_dict[key][['Date','Close']], on="Date")
        index_close_df.rename(columns={'Close':key},inplace=True)

In [18]:
index_close_df.keys()

Index(['Date', 'Nifty_Private_Bank', 'NIFTY_FMCG', 'NIFTY_50', 'NIFTY_AUTO',
       'NIFTY_METAL', 'NIFTY_PHARMA', 'NIFTY_MIDCAP_150', 'NIFTY_BANK',
       'NIFTY_IT', 'NIFTY_NEXT_50', 'NIFTY_OILGAS'],
      dtype='object')

In [19]:
index_dict['NIFTY_50'].head()

,Date,Open,High,Low,Close,Volume
0,2000-01-03,1482.15,1592.90,1482.15,1592.2,25358322
1,2000-01-04,1594.40,1641.95,1594.40,1638.7,38787872
2,2000-01-05,1634.55,1635.50,1555.05,1595.8,62153431
3,2000-01-06,1595.80,1639.00,1595.80,1617.6,51272875
4,2000-01-07,1616.60,1628.25,1597.20,1613.3,54315945


In [20]:
index_dict['NIFTY_50'].tail()

,Date,Open,High,Low,Close,Volume
5488,2022-01-21,17613.70,17707.60,17485.85,17617.15,277645373
5489,2022-01-24,17575.15,17599.40,16997.85,17149.10,323847388
5490,2022-01-25,17001.55,17309.15,16836.80,17277.95,326515896
5491,2022-01-27,17062.00,17182.50,16866.75,17110.15,395596577
5492,2022-01-28,17208.30,17373.50,17077.10,17101.95,355284285


In [21]:
fpi_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, Airlines to Utilities3
Data columns (total 74 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   January152019    45 non-null     object
 1   January312019    45 non-null     object
 2   February152019   45 non-null     object
 3   February282019   45 non-null     object
 4   March152019      45 non-null     object
 5   March312019      45 non-null     object
 6   April152019      45 non-null     object
 7   April302019      45 non-null     object
 8   May152019        45 non-null     object
 9   May312019        45 non-null     object
 10  June152019       45 non-null     object
 11  June302019       45 non-null     object
 12  July152019       45 non-null     object
 13  July312019       45 non-null     object
 14  August152019     45 non-null     object
 15  August312019     45 non-null     object
 16  September152019  45 non-null     object
 17  September302019  45 non-nul

In [22]:
fpi_df.head()

,January152019,January312019,February152019,February282019,March152019,March312019,April152019,April302019,May152019,May312019,...,September152021,September302021,"October15,2021","October31,2021","November15,2021","November30,2021","December15,2021","December31,2021","January15,2022","January31,2022"
Sector,,,,,,,,,,,,,,,,,,,,,
Airlines,6500,7086,6870,6679,7615,8515,8824,9322,9903,10520,...,14633,14508,14986,15721,17108,13654,14017,14486,15516,13143
Airport_Services,0,0,0,0,0,0,0,0,0,0,...,5047,6229,7062,6599,6620,6215,7005,7791,7484,6966
Automobiles_&_Auto_Components,169820,154934,156794,158610,167050,157761,165694,156798,150333,155342,...,193100,198251,212805,202455,210139,193073,197986,197424,207451,209195
Banks,544394,546340,542563,547384,600544,626437,623382,617345,593456,649049,...,858412,875404,916029,907747,893454,818524,831304,802378,866670,849179
Capital_Goods,98627,93736,88096,91977,99262,100021,103004,99834,92612,104685,...,163638,132951,146357,145455,156196,143802,153151,154426,165010,157971


- Now both the data has been converted into the desired format, lets look at the columns.
- In fpi_df,Sector column is set as index and rest of the column consist of Average investment values from 15th January 2019 to 31th January 2022.

In [23]:
index_dict['NIFTY_50'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5493 entries, 0 to 5492
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    5493 non-null   datetime64[ns]
 1   Open    5493 non-null   float64       
 2   High    5493 non-null   float64       
 3   Low     5493 non-null   float64       
 4   Close   5493 non-null   float64       
 5   Volume  5493 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 257.6 KB


In [24]:
index_dict.keys()

dict_keys(['Nifty_Private_Bank', 'NIFTY_FMCG', 'NIFTY_SMALLCAP_250', 'NIFTY_50', 'NIFTY_AUTO', 'NIFTY_METAL', 'NIFTY_PHARMA', 'NIFTY_MIDCAP_150', 'NIFTY_BANK', 'NIFTY_IT', 'NIFTY_NEXT_50', 'NIFTY_OILGAS'])

In [25]:
print(f"List of columns in FPI dataframe: {fpi_df.columns}")

List of columns in FPI dataframe: Index(['January152019', 'January312019', 'February152019', 'February282019',
       'March152019', 'March312019', 'April152019', 'April302019', 'May152019',
       'May312019', 'June152019', 'June302019', 'July152019', 'July312019',
       'August152019', 'August312019', 'September152019', 'September302019',
       'October15,2019', 'October31,2019', 'November152019', 'November302019',
       'December152019', 'December312019', 'January152020', 'January312020',
       'February152020', 'February292020', 'March152020', 'March312020',
       'April152020', 'April302020', 'May152020', 'May312020', 'June152020',
       'June302020', 'July152020', 'July312020', 'August152020',
       'August312020', 'September152020', 'September302020', 'October152020',
       'October312020', 'November152020', 'November302020', 'December152020',
       'December312020', 'January152021', 'January312021', 'February152021',
       'February282021', 'March152021', 'March312021

In [26]:
print(f"List of columns in Nifty_50 dataframe: {index_dict['NIFTY_50'].columns}")

List of columns in Nifty_50 dataframe: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


- Indices data ranges from **3rd Jan 2000 to 28th Jan 2022**.
- The dataset contains over 5493 rows and 6 columns.
- **Open and Close** indicate the opening and closing price of the index on a particular day.
- **High and Low** provide the highest and the lowest price for the index on a particular day, respectively.
- **Volume** indicate the total volume traded on a particular day.
- **Turnover** provide the total value of stocks traded during a specific period of time. The time period may be annually, quarterly, monthly or daily.

Let's now view some basic statistics about the index data frame. 

In [27]:
index_dict['NIFTY_50'].describe()

,Open,High,Low,Close,Volume
count,5493.000000,5493.000000,5493.000000,5493.000000,5.493000e+03
mean,5935.464965,5973.681404,5888.592072,5931.981003,1.927721e+08
std,4060.406773,4073.488083,4036.579030,4055.471502,1.724929e+08
min,853.000000,877.000000,849.950000,854.200000,1.394931e+06
25%,2160.850000,2173.850000,2145.750000,2167.400000,8.232518e+07
50%,5286.600000,5331.800000,5245.500000,5285.000000,1.457869e+08
75%,8537.050000,8588.100000,8494.350000,8530.800000,2.256081e+08
max,18602.350000,18604.450000,18445.300000,18477.050000,1.811564e+09


# Missing values

In [28]:
index_dict['NIFTY_50'].isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [29]:
fpi_df.isnull().sum()

January152019      0
January312019      0
February152019     0
February282019     0
March152019        0
                  ..
November30,2021    0
December15,2021    0
December31,2021    0
January15,2022     0
January31,2022     0
Length: 74, dtype: int64

There are no missing value.

In [30]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "dokeabhishek3/fpi-indices-data-analysis" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis


'https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis'

## Exploratory Analysis and Visualization

Let's begin by importing`plotly`

In [31]:
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

### Visualising the NIFTY 50 close price from Aug 2019  to Jan 2022

In [32]:
nifty_50_2019 = index_dict['NIFTY_50'][index_dict['NIFTY_50']['Date'] >= '2019-08-01']


fig = go.Figure()
fig.add_trace(go.Scatter(x=nifty_50_2019.Date, y=nifty_50_2019.Close,
                        mode='lines', name='Nifty_50'))

fig.update_layout(title='Nifty_50 data 2019-2022',title_x=0.5,
                 xaxis_title='Date', yaxis_title='Price')

# Monthly Total FPI investment

- Calculate sum for each period

In [33]:
total_investment = pd.DataFrame(fpi_df.sum())

In [34]:
total_investment.reset_index(inplace = True)
total_investment.rename(columns = {'index':'Period',0:'total_investment'}, 
            inplace = True)
total_investment['category'] = [str(i) for i in total_investment.index]
color_discrete_sequence = ['#609cd4']*len(total_investment)
# 1st Lockdown 
color_discrete_sequence[29] = '#ec7c34'

# 2nd Lockdown
color_discrete_sequence[54] = '#ec7c34'

fig = px.bar(total_investment, y='total_investment', x='Period',
             color='category',
             color_discrete_sequence=color_discrete_sequence,
             title='Monthly Total FPI investment',
             labels={'total_investment':'Total_Investment', 'Period':'Period'})

fig.update_layout(uniformtext_minsize=8)
fig.update_layout(xaxis_tickangle=-45, showlegend=False)

fig

- The highlighted bars show the announcment of lockdown in India.
- if we comparing both lockdown, we can see that FPI investments decline dramatically in the first lockdown.

# Comparing different sectoral indices

- Lets look at the close price of various sectoral indices.

In [35]:
d = {}
for key in index_dict.keys():
    if (key == 'NIFTY_50')|(key == 'NIFTY_MIDCAP_150')|\
    (key == 'NIFTY_SMALLCAP_250')|(key == 'NIFTY_NEXT_50')|(key == 'NIFTY_OILGAS'):
        pass
    else:
        temp = index_dict[key][index_dict[key]['Date'] >= '2019-08-01']
        d[f'{key} index'] = temp['Close'].values

indices_df = pd.DataFrame(data=d)
indices_df.index=nifty_50_2019['Date']

In [36]:
fig = indices_df.iplot(asFigure=True,hline=[0,4], vline=['2020-03-23','2021-04-15'], )

fig.update_layout(title='NIFTY INDICES',title_x=0.5,
                  yaxis_title='Close Price',
                  xaxis_title='Periods',
                  annotations=[dict(x='2020-03-23', y=0.95, xref='x', yref='paper',
                    showarrow=False, xanchor='right', text='First Lockdown'),
                              dict(x='2021-04-15', y=0.95, xref='x', yref='paper',
                    showarrow=False, xanchor='right', text='Second Lockdown')],
)
fig.show()

- Now lets from total investment to sector wise fund allocation

- Out of total 45 sectors we will look at those sectors where major changes took place.

# Sectorwise Fund Allocation

- Out of total 45 sectors we will compare only those sectors which showed a significant change in fund allocation. 

In [37]:
# Set sector column as index 
# transpose the whole dataframe and then reset the index
fpi_avg_df_t = fpi_avg_df.set_index('Sector').T.reset_index()

# removing underscore from the string and fetch only the date part 
fpi_avg_df_t['index'] = fpi_avg_df_t['index'].apply(lambda x: x.split('_')[-1])

# removing extra character from the string
try:
    fpi_avg_df_t['index']= fpi_avg_df_t['index'].apply(lambda x: x.replace(",",""))
except:
    pass

# Parsing strings into datetimes object 
fpi_avg_df_t['index'] = fpi_avg_df_t['index'].apply(lambda x: datetime.datetime.strptime(x,'%B%d%Y').date()) 
fpi_avg_df_t.rename(columns={'index':'Date'}, inplace = True)

In [38]:
fig = px.bar(fpi_avg_df_t.iloc[:,:], x='Date', y=['Banks','Other_Financial_Services1',
                                          'Total_Financial_Services','Software_&_Services','Oil_&_Gas',
                                          'Telecom_Services','Utilities3'],
      barmode='group',title='Sector wise Fund Allocation')
fig.update_layout(showlegend=False)
fig

# Overview of Fund allocation for the month of January 2022

In [39]:
fig=px.pie(fpi_df['January31,2022'],
           names=fpi_df['January31,2022'].index,
           values=fpi_df['January31,2022'],
      title='January Fund Allocation',
      color_discrete_sequence=px.colors.sequential.RdBu)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig

# HeatMap to check correlation between variables

In [40]:
common_df =index_close_df.copy()
common_df.set_index('Date', inplace=True)
common_df=common_df.rolling(window=30).mean()
common_df = common_df[common_df.index > '2018-02-12']

In [41]:
temp_fpi_df = fpi_avg_df_t.copy()

# handle saturdays and sunday and moving data ahead by 3 days 
bd = pd.tseries.offsets.BusinessDay(n = 3)
temp_fpi_df['Date'] = temp_fpi_df['Date'] + bd
temp_fpi_df.at[temp_fpi_df[temp_fpi_df.Date=='2019-06-05'].index.values[0],'Date']='2019-06-06'
temp_fpi_df.set_index('Date', inplace=True)

#  adding columns from fpi data 
common_df = common_df.reindex(columns = common_df.columns.tolist()+temp_fpi_df.columns.tolist())

# updating values from fpi to merged data 
common_df.update(temp_fpi_df)

In [42]:
# subseting data from 2019-04-02 because we have limited fpi data
common_df = common_df[common_df.index>'2019-04-02']

In [43]:
common_df = common_df.ffill(axis=0)

# Removing columns with zero values
common_df = common_df.loc[:,(common_df !=0).any(axis=0)]

In [44]:
fig = px.imshow(common_df.corr(),color_continuous_scale='Viridis_r',
               )
fig.update_xaxes(side='top',
        tickangle = -50)

fig.layout.height=800
fig.layout.width=800
fig.show()

# Heatmap on Indices data 

In [45]:
fig = px.imshow(index_close_df[index_close_df.columns[1:]].corr(),color_continuous_scale='Viridis_r',
               )
# fig.update_traces(reversescale=False)
fig.update_xaxes(
        tickangle = -50)
fig.layout.height=800
fig.layout.width=800               
fig.show()

# Compare FPIs Oil & Gas sector data and Nifty_OILGAS 

In [46]:
fig = go.Figure()
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=common_df.index, y=common_df['Oil_&_Gas'],
                        mode='lines', name='FPIs Oil_&_Gas'),secondary_y=True)

fig.add_trace(go.Scatter(x=common_df.index, y=common_df.NIFTY_OILGAS,
                        mode='lines', name='NIFTY OILGAS'))

fig.show()

# OLS Slope

In [47]:
fig = px.scatter(common_df[['NIFTY_OILGAS','Oil_&_Gas']],x="Oil_&_Gas", y="NIFTY_OILGAS",
                trendline="ols")
fig.show()

In [48]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "dokeabhishek3/fpi-indices-data-analysis" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis


'https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis'

## Asking and Answering Questions

#### Q1: Which sectors is displaying a positive shift as no 30th of September 2021, and do the sectorial indices reflect this?

In [49]:
fpi_avg_df[['Sector','Average_for_September302021']][fpi_avg_df['Average_for_September302021']>0].sort_values(by='Average_for_September302021',ascending=False).head()

,Sector,Average_for_September302021
35,Software_&_Services,27016.50
41,Total_Financial_Services,20154.00
27,Other_Financial_Services1,16466.25
26,Oil_&_Gas,15607.50
44,Utilities3,7154.75


In [50]:
fig = px.bar(fpi_avg_df_t.iloc[-15:,:], x='Date', y=['Software_&_Services'],
      barmode='group',title='Fund Allocation in Software & Services')
fig.update_layout(showlegend=False)
fig

In [51]:
nifty_IT = index_dict['NIFTY_IT'][index_dict['NIFTY_IT']['Date']>'2021-07-1']

fig = go.Figure()
fig.add_trace(go.Scatter(x=nifty_IT.Date, y=nifty_IT.Close,
                        mode='lines+markers', name='Nifty IT'))

fig.update_layout(title='Nifty_IT from September 2021 to January 2022',
                 xaxis_title='Date', yaxis_title='Price')

fig.show()

- Check correlation with the two

In [52]:
common_df[['NIFTY_IT','Software_&_Services']].corr()

,NIFTY_IT,Software_&_Services
NIFTY_IT,1.000000,0.366954
Software_&_Services,0.366954,1.000000


#### OLS Slope for entire data

In [53]:
fig = px.scatter(common_df[['NIFTY_IT','Software_&_Services']], x="Software_&_Services", y="NIFTY_IT",
                trendline="ols")
fig

- The IT industry saw a net inflow of about 27,000 crore between July and September 2021 and the same can be observed in the nifty IT index.


- There is a positive correlation between the two variables and even the OLS plot shows a positive slope.

#### Q2: Which sectors showed a positive change during first phase of lockdown?

In [54]:
fpi_avg_df_t['Date'] = pd.to_datetime(fpi_avg_df_t['Date'], format='%Y-%m-%d')
first_phase =fpi_avg_df_t[(fpi_avg_df_t['Date']>'2020-03-31') & (fpi_avg_df_t['Date']<'2020-05-31')]

In [55]:
positive_sector =pd.DataFrame((first_phase[first_phase.columns[1:]]>0).any())

In [56]:
positive_sector[positive_sector[0]>0].index

Index(['Household_&_Personal_Products', 'Oil_&_Gas',
       'Pharmaceuticals_&_Biotechnology'],
      dtype='object', name='Sector')

In [57]:
first_phase[['Date','Household_&_Personal_Products','Oil_&_Gas','Pharmaceuticals_&_Biotechnology']]

Sector,Date,Household_&_Personal_Products,Oil_&_Gas,Pharmaceuticals_&_Biotechnology
25,2020-04-15,-1073.75,-18313.75,-3785.00
26,2020-04-30,948.75,-19359.00,-263.50
27,2020-05-15,698.50,1198.75,3070.25


In [58]:
fig = px.bar(first_phase, x='Date', y=['Household_&_Personal_Products','Oil_&_Gas','Pharmaceuticals_&_Biotechnology'],
      barmode='group',title='First Phase of Lockdown')
fig

- Only three sectors exhibited a positive shift at the end of the first phase of lock-down, as seen in the bar chart.

#### Q3: What was the scenario in Pharma sector before and after the announcement of lock down?

In [59]:
before_lockdown = fpi_avg_df_t[(fpi_avg_df_t['Date']>'2019-06-30') & (fpi_avg_df_t['Date']<'2020-09-30')]
nifty_Pharma = index_dict['NIFTY_PHARMA'][(index_dict['NIFTY_PHARMA']['Date']>'2019-07-15') &
                                         (index_dict['NIFTY_PHARMA']['Date']<'2020-09-30')]

fig = px.bar(before_lockdown, x='Date', y='Pharmaceuticals_&_Biotechnology')

fig.add_trace(go.Scatter(x=nifty_Pharma.Date, y=nifty_Pharma.Close,
                        mode='lines', name='Nifty Pharma'))

fig.update_layout(title='Nifty_Pharma vs FPI Investment',
                 xaxis_title='Date', yaxis_title='Price')


fig.show()

- Till November 2019 Pharmaceuticals & Biotechnology FPIs have been net sellers for consecutive months.


- It is interesting to see, From December onwards FPIs started pumping in Pharma sector but nifty_pharma index was moving sideways.


- After the first phase of lockdown both the data is showing an up trend.

#### Q4: Considering the limited FPI data,  which sector have shown significant changes ?

In [60]:
fig = px.bar(fpi_avg_df_t, x='Date', y=['Total_Financial_Services'],
      barmode='group')
fig

In [61]:
nifty_pvt_bank = index_dict['Nifty_Private_Bank'][index_dict['Nifty_Private_Bank']['Date']>'2019-03-31'][['Date','Close']]
nifty_pvt_bank.rename(columns={'Close':'Nifty_Private_Bank'},inplace=True)
nifty_bank = index_dict['NIFTY_BANK'][index_dict['NIFTY_BANK']['Date']>'2019-03-31'][['Date','Close']]
nifty_bank.rename(columns={'Close':'NIFTY_BANK'},inplace=True)
finance = pd.merge(nifty_pvt_bank, nifty_bank, on="Date")
finance.set_index('Date', inplace=True)


fig = go.Figure()
fig.add_trace(go.Scatter(x=finance.index, y=finance.Nifty_Private_Bank,
                        mode='lines', name='Nifty Private Bank'))

fig.add_trace(go.Scatter(x=finance.index, y=finance.NIFTY_BANK,
                        mode='lines', name='Nifty Bank'))


fig.show()


In [62]:
common_df[['Nifty_Private_Bank','NIFTY_BANK','Total_Financial_Services']].corr()

,Nifty_Private_Bank,NIFTY_BANK,Total_Financial_Services
Nifty_Private_Bank,1.000000,0.986545,0.378344
NIFTY_BANK,0.986545,1.000000,0.310331
Total_Financial_Services,0.378344,0.310331,1.000000


In [63]:
fig = go.Figure()
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=common_df.index, y=common_df.Total_Financial_Services,
                        mode='lines', name='Financial Services'),secondary_y=True)

fig.add_trace(go.Scatter(x=common_df.index, y=common_df.Nifty_Private_Bank,
                        mode='lines', name='Nifty Private Bank'))

fig.add_trace(go.Scatter(x=common_df.index, y=common_df.NIFTY_BANK,
                        mode='lines', name='Nifty Bank'))

fig.show()

- Looking at sector wise fund allocation's bar plot we can conclude that financial_service sector has shown significant change in fund allocation by FPIs.


- When ever there is a pump in or pull back of funds by FPIs the same can be seen in the respective indices line chart.

#### Q5: Which sector has never benefited from foreign investment? 

In [64]:
positive_sector =pd.DataFrame((fpi_avg_df_t[fpi_avg_df_t.columns[1:]]==0).any())
positive_sector[positive_sector[0]==True].index.tolist()

['Airport_Services',
 'Diversified_Consumer_Services',
 'Food_&_Drugs_Retailing',
 'Hardware_Technology_&_Equipment',
 'Real_Estate_Investment',
 'Sovereign',
 'Surface_Transportation',
 'Telecommunications_Equipment']

- FPIs have made no investment in the sectors listed above.

#### Q5:Calculate correlation between the indices and FPI report

In [65]:
common_df.corr().iloc[11:,:11]

,Nifty_Private_Bank,NIFTY_FMCG,NIFTY_50,NIFTY_AUTO,NIFTY_METAL,NIFTY_PHARMA,NIFTY_MIDCAP_150,NIFTY_BANK,NIFTY_IT,NIFTY_NEXT_50,NIFTY_OILGAS
Airlines,0.258542,0.251155,0.246701,0.347811,0.206681,0.323048,0.225364,0.217705,0.217631,0.227380,0.250706
Airport_Services,0.374654,0.328160,0.345190,0.380825,0.289135,0.189091,0.321443,0.381146,0.265607,0.337648,0.393524
Automobiles_&_Auto_Components,0.137439,0.173536,0.156809,0.296235,0.040144,0.243128,0.112111,0.109413,0.110975,0.135761,0.171586
Banks,0.302437,0.151049,0.161400,0.274818,0.062383,0.158576,0.093997,0.234583,0.069339,0.118688,0.138278
Capital_Goods,0.064592,-0.006870,0.024713,0.183506,-0.105385,0.139717,-0.024877,0.011792,0.002791,-0.018732,-0.006402
Chemicals_&_Petrochemicals,0.302261,0.286060,0.267870,0.391654,0.291009,0.428906,0.244353,0.269243,0.189522,0.253814,0.261495
Coal,0.429667,0.247662,0.250261,0.377479,0.257179,0.169847,0.212825,0.370179,0.140624,0.228496,0.293449
Commercial_Services_&_Supplies,0.242721,0.168882,0.234407,0.362810,0.226444,0.365553,0.225937,0.231266,0.191569,0.211217,0.200510
Construction_Materials,0.347337,0.270747,0.248076,0.339030,0.194257,0.337975,0.190855,0.289104,0.168601,0.208467,0.191076
Consumer_Durables,0.358070,0.490764,0.415569,0.411662,0.396695,0.526002,0.382922,0.353891,0.380972,0.403060,0.370666


## Conclusion

- In this project we have compared FPIs data and Nifty indices data.


- It's worth noting that there's a delay in receiving the FPI Report; it's the 2nd of March,2022 as I write this notebook, and the report for February 28th 2022 is still unavailable.


- It was interesting to examine the pre and post lockdown scenarios,since the pharma sector showed a fund inflow as per FPIs data ,while the nifty pharma index displayed consolidation and following the first phase of lock-down, the pharma index began to rise.


- We also analysed financial data,which demonstrated a high correlation between Nifty Bank and Financial sector.


- As a result, we can conclude that FPI data can assist us in detecting early indications and can be used to get a better understanding of the Indian market before investing.
 


# Ideas for future works
- I have manually imported the FPI data from nsdl using google sheets and downloaded it as xlsx format, instead we can pharse the table using web scraping python library i.e Beautiful Soup or Selenium and maintain a SQL database.


- Further we can use FPIs data as one of the feature in machine learning model to backtest a trading strategy.


- We can also create an interactive dashboard using plotly and deploy it using streamlit or Django.

## References

- [Data Analysis with Python: Zero to Pandas] (https://jovian.ml/learn/data-analysis-with-python-zero-to-pandas)
-[Nifty Indices Data] (https://www.kaggle.com/atrisaxena/nifty-indices-data)
-[Plotly] (https://plotly.com/)
-[Pandas] (https://pandas.pydata.org/pandas-docs/stable/index.html)

In [66]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "dokeabhishek3/fpi-indices-data-analysis" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis


'https://jovian.ai/dokeabhishek3/fpi-indices-data-analysis'